In [4]:
# acount metadata
AZURE_SUBSCRIPTION_ID = 'ef3e36f3-4579-469d-b38e-9812ed58ed4f'
ML_WORKSPACE_NAME = 'dairyDemo'
AZURE_IOT_HUB_NAME = 'dairyFarm'
RESOURCE_GROUP_NAME = 'IoTEdgeResources'
LOCATION = 'West US 2'
STORAGE_ACCOUNT_NAME = 'dairydatastorage'
STORAGE_ACCOUNT_KEY = 'oJ7P/zbnlwzxGHNTVar6eaEU9ZAk+gvM3YrPRKeqZAU5ymrMH4yilNkQP6baAoNi28yi9SwcWCMavRPGIb2phw=='
STORAGE_ACCOUNT_CONTAINER = 'dairydata'
DATASTORE_NAME = 'dairydata'

In [ ]:
import os, glob, shutil
from azureml.core import Workspace
## Configue workspace
# from azureml.core.workspace import Workspace
# ws = Workspace.from_config()
workspace_name = ML_WORKSPACE_NAME
subscription_id = AZURE_SUBSCRIPTION_ID
resource_group = RESOURCE_GROUP_NAME
location = LOCATION

if not os.path.exists('./aml_config'):
    os.mkdir('./aml_config')
ws = Workspace.from_config()

In [5]:
## print workspace detail

import azureml.core
import pandas as pd
from azureml.core import Workspace

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data=output, index=['']).T

,
SDK version,1.0.72
Subscription ID,ef3e36f3-4579-469d-b38e-9812ed58ed4f
Workspace,dairyDemo
Resource Group,IoTEdgeResources
Location,westus2


In [16]:
## retrieve data from datastore, create TabularDataset 
from azureml.core import Datastore, Workspace, Datastore, Dataset

datastore_name = DATASTORE_NAME
datastore = Datastore.get(ws, datastore_name)
label_datastore_path = (datastore,'/label_data_large.csv')
label_dataset = Dataset.Tabular.from_delimited_files(path=label_datastore_path)

record_datastore_path_old = (datastore,'/record_data_large.csv')
record_dataset_old = Dataset.Tabular.from_delimited_files(path=record_datastore_path_old)
record_datastore_path = (datastore,'/dairy_milk_daily_data.csv')
record_dataset = Dataset.Tabular.from_delimited_files(path=record_datastore_path)

# preprocessed  
full_data_path = record_datastore_path = (datastore,'/preprocessed_data.csv')
full_dataset = Dataset.Tabular.from_delimited_files(path=full_data_path)
full_data = full_dataset.to_pandas_dataframe()

print("Data retrieved!")

Data retrieved!


In [17]:
full_data.head()

,Column1,datesql,Animal_ID,Activity(steps/hr),ActivityDeviation(%),Protein(%),Fat(%),Lactose,Conductivity,Blood(%),...,AnimalStatus,Abortion,Fresh (Calving),Heat,Insemination,Not for Insemination,Pregnant,IsEstrus,R,Event
0,0,2018-01-03,1,95.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,,
1,1,2018-01-03,2,166.0,13.0,0.0,0.0,0.0,9.0,0.0,...,1,0,0,0,0,0,1,0,,
2,2,2018-01-03,3,163.0,18.0,0.0,0.0,0.0,9.1,0.0,...,1,0,0,0,0,0,1,0,,
3,3,2018-01-03,10,214.0,13.0,0.0,0.0,0.0,9.1,0.0,...,1,0,0,1,0,0,0,0,,
4,4,2018-01-03,18,154.0,45.0,0.0,0.0,0.0,9.4,0.0,...,1,0,0,0,0,0,1,0,,


### Columns and Explanation 
##### Explaining columns 
Four set of effecting variables:  cows' health, activity, milk composition; 

some of health_cols are formed from categorical data
1. ['Abortion', 'Fresh (Calving)', 'Heat', 'Insemination', 'Not for Insemination','Pregnant'] : are one-hot encoding from "Gynecology_Status" 
2. ["AnimalStatus"]  :  {0: "Dry", 1: "Milk"}

##### training models: 
(it would be great if we could have 2 sets of models with different variables to compare)

only using [health cols] ( only using [activity cols]), combining [health, activity, milk]       
(or other combination you think is better)

In [9]:
cow_cols = ["datesql",  "Animal_ID"]
health_cols = ['Blood(%)','Avg_Milking_Time(seconds)','RestBout(#)','RestRatio(%)','RestTime(min)','Weight(gr)',"AnimalStatus",'Abortion', 'Fresh (Calving)', 'Heat', 'Insemination', 'Not for Insemination','Pregnant']
activity_cols =['Activity(steps/hr)','ActivityDeviation(%)']
milk_cols = ['Protein(%)',"Fat(%)",'Lactose','Conductivity']
labels = ['R','Event','IsEstrus']

###  Upsample and balance minority group
these functions could be adjusted based on your training needs

In [22]:
from sklearn.utils import resample

# separate minority and majority classes
X = full_data
not_estrus = X[X.IsEstrus==0]
estrus = X[X.IsEstrus==1]

# upsample minority
estrus_upsampled = resample(estrus,replace=True, # sample with replacement
                        n_samples=len(not_estrus), # match number in majority class
                          random_state=27) # reproducible results
# combine majority and upsampled minority
upsampled = pd.concat([not_estrus, estrus_upsampled])

# check new class counts
upsampled.IsEstrus.value_counts()
y_train = upsampled.pop('IsEstrus').to_frame()
X_train = upsampled

X_train.shape;
sum(y_train.IsEstrus)

53505

## Training ML models

## Create Docker Image
#### copy from estrus_xh.ipbny

In [24]:
%%writefile estr.py
# This script generates the  file
# with the init and run functions needed to 
# operationalize the estrus prediction

import pickle
import json
import pandas
from sklearn.externals import joblib
from sklearn.linear_model import Ridge
from azureml.core.model import Model


def init():
    global model
    # this is a different behavior than before when the code is run locally, even though the code is the same.
    model_path = Model.get_model_path('model.pkl')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
    
input_sample = pd.DataFrame(data=[{
    "Activity(steps/hr)": 186.0,
    "ActivityDeviation(%)": -11.0,
    "Yield(gr)": 11226.0,
    "YieldDeviation(%)": -15.0,
    "Fat(%)": 4.09,
    "FatDeviation(%)": 7.0,
    "Protein(%)": 2.64,
    "ProteinDeviation(%)": -8.0,
    "Lactose(%)": 4.96,
    "LactoseDeviation(%)": 3.0,
    "Conductivity": 8.3,
    "ConductivityDeviation(%)": -2.0,
    "SCC (*1000/ml)": 26363.0,
    "Blood(%)": 0.03,
    "ProductionRate(gr/hr)": 1528.0
}])
output_sample = np.array([0.9974, 0.0026])

# note you can pass in multiple rows for scoring
def run(input_str):
    try:
        result = model.predict(data)
        # You can return any data type, as long as it is JSON serializable.
        print("Prediction is ", result[0])
        return result.tolist()
    except Exception as e:
        result = str(e)
        
    if pred[0] == 1:
        input_json['anomaly']=True
    else:
        input_json['anomaly']=False
        
    return [json.dumps(input_json)]

Overwriting estr.py


In [25]:
# This specifies the dependencies to include in the environment
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(conda_packages=['azureml-defaults','azureml-train-automl','inference-schema[numpy-support]','joblib',
    'numpy',
    'scikit-learn==0.20.2',
    'pandas'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [27]:
# Add tags to the images
from azureml.core.image import Image, ContainerImage

MODEL_NAME = "estruspred_new"

image_config = ContainerImage.image_configuration(runtime= "python",execution_script="estr.py",
                                 conda_file="myenv.yml",tags = {'area': "iot", 'type': "classification"},
                                 description = "Estrus prediction model")

image = Image.create(name = MODEL_NAME,
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

In [ ]:
image.wait_for_creation(show_output = True)

In [ ]:
for i in Image.list(workspace = ws,tags = ["area"]):
    print('{}(v.{} [{}]) stored at {} with build log {}'.format(i.name, i.version, i.creation_state, i.image_location, i.image_build_log_uri))

## Test image

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'area': "iot", 'type': "classification"}, 
                                               description = 'estrus edge prediction')

In [ ]:
## Test the deployed service
import requests
import json
from azureml.core.authentication import InteractiveLoginAuthentication

# Get a token to authenticate to the compute instance from remote
interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

# Create and submit a request using the auth header
headers = auth_header

# Add content type header
headers.update({'Content-Type':'application/json'})

# Create test sample
test_sample = json.dumps({'data': [
    {
    "Activity(steps/hr)": 186.0,
    "ActivityDeviation(%)": -11.0,
    "Yield(gr)": 11226.0,
    "YieldDeviation(%)": -15.0,
    "Fat(%)": 4.09,
    "FatDeviation(%)": 7.0,
    "Protein(%)": 2.64,
    "ProteinDeviation(%)": -8.0,
    "Lactose(%)": 4.96,
    "LactoseDeviation(%)": 3.0,
    "Conductivity": 8.3,
    "ConductivityDeviation(%)": -2.0,
    "SCC (*1000/ml)": 26363.0,
    "Blood(%)": 0.03,
    "ProductionRate(gr/hr)": 1528.0
}
]})
test_sample = bytes(test_sample,encoding = 'utf8')

# Send request
##   URL   ACESSS AND RUN  FOR FRONT END    ------------------*************---------------------
service_url = 'http://c7268248-3d6a-409f-87b7-e3b8429325d0.westus2.azurecontainer.io/score'
resp = requests.post(service_url, test_sample, headers=headers)
print(resp.text)

## Deploy container to Azure IoT Edge Device

In [ ]:
# Update the workspace object
ws = Workspace.from_config()

# Getting your container details
container_reg = ws.get_details()["containerRegistry"]
reg_name=container_reg.split("/")[-1]
container_url = "\"" + image.image_location + "\","
subscription_id = ws.subscription_id
print('{}'.format(image.image_location))
print('{}'.format(reg_name))
print('{}'.format(subscription_id))
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azure.mgmt import containerregistry
client = ContainerRegistryManagementClient(ws._auth,subscription_id)
result= client.registries.list_credentials(RESOURCE_GROUP_NAME, reg_name, custom_headers=None, raw=False)
username = result.username
password = result.passwords[0].value

In [ ]:
file = open('estrus-model-deployment-template.json')
contents = file.read()
contents = contents.replace('__MODULE_NAME', MODEL_NAME)
contents = contents.replace('__REGISTRY_NAME', reg_name)
contents = contents.replace('__REGISTRY_USER_NAME', username)
contents = contents.replace('__REGISTRY_PASSWORD', password)
contents = contents.replace('__REGISTRY_IMAGE_LOCATION', image.image_location)
with open('./deployment.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)